# Creating a Simple Agent with Tracing

In [1]:
import dotenv
import os

from openai import OpenAI

dotenv.load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    print(
        """Error: OPENAI_API_KEY environment variable not set. Please copy the .env.template file as .env and fill it in.
    
    You can execute these commands in the terminal to get started:
    cp .env.template .env
    code .env
    """
    )

# Test OpenAI Access
print(
    OpenAI()
    .responses.create(
        model=os.environ["OPENAI_DEFAULT_MODEL"], input="Say: We are up and running!"
    )
    .output_text
)

We are up and running!


In [2]:
from agents import Agent, Runner, trace
from openai.types.responses import ResponseTextDeltaEvent

Create a simple Nutrition Assistant Agent

In [3]:
nutrition_agent = Agent(
    name="Nutrition Agent",
    instructions="" \
    "You are a helpful assistant giving out nutrition advice. You give concise answers")

Let's execute the Agent:

In [7]:
with trace("Simple Nutrition Agent"):
    result = await Runner.run(nutrition_agent, "How healthy are bananas?")

print(result)

RunResult:
- Last agent: Agent(name="Nutrition Agent", ...)
- Final output (str):
    - Nutrients: good source of potassium, vitamin B6, vitamin C, and dietary fiber.
    - Health benefits: supports heart health, digestion, and steady energy; contains resistant starch when less ripe.
    - Considerations: higher sugar content as bananas ripen; moderate portions fit most diets.
    - When to limit: if you have kidney disease or need potassium restrictions; if you have diabetes, monitor portion and pair with protein/fat to blunt sugar spike.
    - Quick tips: 1–2 bananas per day is common; choose ripeness by preference (green for less sugar, yellow-brown for more sweetness).
- 2 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


Streaming the answer to the screen, token by token

In [8]:
response_stream = Runner.run_streamed(nutrition_agent, "How healthy are bananas?")

async for event in response_stream.stream_events():
    if event.type == "raw_response_event" and isinstance(
        event.data, ResponseTextDeltaEvent
    ):
        print(event.data.delta, end="", flush=True)

Generally healthy as part of a balanced diet.

Key nutrients per medium banana (~118 g):
- Potassium ~422 mg
- Vitamin B6, vitamin C
- 3 g fiber, ~14 g sugar, ~105 calories
- Low in fat

Benefits:
- Supports heart health, digestion, and energy.

Tips:
- Pair with protein or fats (e.g., yogurt, peanut butter) to slow sugar absorption.
- If watching carbs/sugar (diabetes), enjoy in moderation or choose smaller/less ripe bananas (less sugar when less ripe? Actually less ripe has more starch; riper has more sugar; choose portion accordingly).

Overall: a convenient, nutrient-dense fruit for most people.

_Good Job!_